<a href="https://colab.research.google.com/github/joaopauloribeirao-jpg/NotebookLM/blob/main/Controle_de_Vendas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
language = 'pt'


# 1. Gravação de Áudio Com Python (e Uma Pitada de JavaScript) 🎤

In [28]:
from IPython.display import Audio, display, Javascript
from google.colab import output
from base64 import b64decode

RECORD = """
const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""

def record(sec=5):
  display(Javascript(RECORD))
  js_result = output.eval_js('record(%s)' % (sec * 1000))
  audio = b64decode(js_result.split(',')[1])

  # >>> CAMINHO FIXO E CONSISTENTE <<<
  file_name = "/content/request_audio.wav"

  with open(file_name, 'wb') as f:
    f.write(audio)

  print(f"Arquivo salvo em: {file_name}")
  return file_name

print('Ouvindo...\n')
record_file = record(6)   # grava 6 segundos

display(Audio(record_file, autoplay=False))


Ouvindo...



<IPython.core.display.Javascript object>

Arquivo salvo em: /content/request_audio.wav


# 2. Reconhecimento de Fala com Whisper (OpenAI) 🧠

In [29]:
!pip install git+https://github.com/openai/whisper.git -q

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [30]:
import whisper

print("Carregando Whisper...")
model = whisper.load_model("small")

# Garante que language existe
language = "pt"

# Transcreve o MESMO arquivo gravado
result = model.transcribe(record_file, fp16=False, language=language)

transcription = result["text"]

print(transcription)


Carregando Whisper...
 Faturamento total.


# 3. Total de Vendas (OpenAI) 🧠

In [31]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [32]:
import pandas as pd

df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/vendas_simuladas.csv")
df.head()

,mes,vendedor,valor
0,Jan,João da Dio,20000
1,Jan,Maria da Dio,17000
2,Jan,Carlos da Dio,13000
3,Fev,João da Dio,20000
4,Fev,Maria da Dio,17000


In [33]:
import pandas as pd

# ===============================
# 1) GARANTIR ORDEM CORRETA DOS MESES
# ===============================

ordem_meses = ["Jan","Fev","Mar","Abr","Mai","Jun",
               "Jul","Ago","Set","Out","Nov","Dez"]

df["mes"] = pd.Categorical(df["mes"], categories=ordem_meses, ordered=True)

# ===============================
# 2) MÉTRICAS BÁSICAS (BASE DE TUDO)
# ===============================

# Total geral do ano
faturamento_total = df["valor"].sum()

# Total por vendedor no ano (ordenado do maior para o menor)
total_por_vendedor = (
    df.groupby("vendedor")["valor"]
      .sum()
      .sort_values(ascending=False)
)

# Total por mês (todos vendedores juntos)
total_por_mes = (
    df.groupby("mes")["valor"]
      .sum()
)

# ===============================
# 3) VENDAS POR VENDEDOR EM CADA MÊS  ✅ (FUNDAMENTAL PARA SUAS PERGUNTAS)
# ===============================

vendas_por_vendedor_mes = (
    df.pivot_table(index="mes",
                   columns="vendedor",
                   values="valor",
                   aggfunc="sum")
)

# ===============================
# 4) TRIMESTRES
# ===============================

mapa_trimestre = {
    "Jan":"Q1","Fev":"Q1","Mar":"Q1",
    "Abr":"Q2","Mai":"Q2","Jun":"Q2",
    "Jul":"Q3","Ago":"Q3","Set":"Q3",
    "Out":"Q4","Nov":"Q4","Dez":"Q4"
}

df["trimestre"] = df["mes"].map(mapa_trimestre)

# Total geral por trimestre
total_por_trimestre = (
    df.groupby("trimestre")["valor"]
      .sum()
)

# Total por vendedor por trimestre
total_vendedor_trimestre = (
    df.groupby(["vendedor","trimestre"])["valor"]
      .sum()
)

# ===============================
# 5) BIMESTRES
# ===============================

mapa_bimestre = {
    "Jan":"B1","Fev":"B1",
    "Mar":"B2","Abr":"B2",
    "Mai":"B3","Jun":"B3",
    "Jul":"B4","Ago":"B4",
    "Set":"B5","Out":"B5",
    "Nov":"B6","Dez":"B6"
}

df["bimestre"] = df["mes"].map(mapa_bimestre)

# Total geral por bimestre
total_por_bimestre = (
    df.groupby("bimestre")["valor"]
      .sum()
)

# Total por vendedor por bimestre
total_vendedor_bimestre = (
    df.groupby(["vendedor","bimestre"])["valor"]
      .sum()
)

# ===============================
# 6) SEMESTRES
# ===============================

mapa_semestre = {
    "Jan":"S1","Fev":"S1","Mar":"S1",
    "Abr":"S1","Mai":"S1","Jun":"S1",
    "Jul":"S2","Ago":"S2","Set":"S2",
    "Out":"S2","Nov":"S2","Dez":"S2"
}

df["semestre"] = df["mes"].map(mapa_semestre)

# Total geral por semestre
total_por_semestre = (
    df.groupby("semestre")["valor"]
      .sum()
)

# Total por vendedor por semestre
total_vendedor_semestre = (
    df.groupby(["vendedor","semestre"])["valor"]
      .sum()
)

# ===============================
# 7) COMPARAÇÕES (QUEM MAIS / QUEM MENOS)
# ===============================

# Quem mais vendeu em cada mês
mais_vendeu_por_mes = (
    df.loc[df.groupby("mes")["valor"].idxmax()][["mes","vendedor","valor"]]
)

# Quem menos vendeu em cada mês
menos_vendeu_por_mes = (
    df.loc[df.groupby("mes")["valor"].idxmin()][["mes","vendedor","valor"]]
)

# ===============================
# 8) "MELHORES" VISÕES (RANKINGS)
# ===============================

# Melhor vendedor do ano
melhor_ano = total_por_vendedor.idxmax()

# Melhor vendedor por trimestre
melhor_por_trimestre = (
    df.groupby(["trimestre","vendedor"])["valor"]
      .sum()
      .groupby("trimestre")
      .idxmax()
)

# Melhor vendedor por mês (tabela pronta)
melhor_por_mes = (
    df.loc[df.groupby("mes")["valor"].idxmax()][["mes","vendedor","valor"]]
)

# ===============================
# 9) PACOTE FINAL DE RESULTADOS (PARA ENVIAR AO CHATGPT DEPOIS) ✅ (CORRIGIDO)
# ===============================

contexto_dados = {
    "faturamento_total": faturamento_total,
    "total_por_vendedor": total_por_vendedor.to_dict(),
    "total_por_mes": total_por_mes.to_dict(),
    "total_por_trimestre": total_por_trimestre.to_dict(),
    "total_por_semestre": total_por_semestre.to_dict(),
    "total_por_bimestre": total_por_bimestre.to_dict(),
    "vendas_por_vendedor_mes": vendas_por_vendedor_mes.to_dict(),  # <-- ESSENCIAL
    "melhor_vendedor_ano": melhor_ano,
}

print("=== RESUMO ANALÍTICO PRONTO ===")
print("Faturamento total:", faturamento_total)
print("Total por vendedor:", total_por_vendedor.to_dict())
print("Total por mês:", total_por_mes.to_dict())


=== RESUMO ANALÍTICO PRONTO ===
Faturamento total: 600000
Total por vendedor: {'João da Dio': 230000, 'Maria da Dio': 209000, 'Carlos da Dio': 161000}
Total por mês: {'Jan': 50000, 'Fev': 50000, 'Mar': 50000, 'Abr': 50000, 'Mai': 50000, 'Jun': 50000, 'Jul': 50000, 'Ago': 50000, 'Set': 50000, 'Out': 50000, 'Nov': 50000, 'Dez': 50000}


/tmp/ipython-input-573261225.py:28: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby("mes")["valor"]
/tmp/ipython-input-573261225.py:37: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  df.pivot_table(index="mes",
/tmp/ipython-input-573261225.py:126: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.loc[df.groupby("mes")["valor"].idxmax()][["mes","vendedor","valor"]]
/tmp/ipython-input-573261225.py:131: FutureWarning: The default of observed=Fa

# 4. Integração com a API do ChatGPT 💬

In [34]:
!pip install openai

In [35]:
import os

# Documentação Oficial da API OpenAI: https://platform.openai.com/docs/api-reference/introduction
# Informações sobre o Período Gratuito: https://help.openai.com/en/articles/4936830

# Para gerar uma API Key:
# 1. Crie uma conta na OpenAI
# 2. Acesse a seção "API Keys"
# 3. Clique em "Create API Key"
# Link direto: https://platform.openai.com/account/api-keys

# Substitua o texto "TODO" por sua API Key da OpenAI, ela será salva como uma variável de ambiente.
os.environ['OPENAI_API_KEY'] = 'TESTE'

In [36]:
# ===============================
# VERSÃO LOCAL (SEM OPENAI) - AJUSTADA (CORRIGIDA)
# ===============================

def perguntar_chatgpt(transcription, contexto_dados):
    """
    Simula o comportamento do ChatGPT sem precisar de API Key.
    Responde APENAS às 3 perguntas solicitadas.
    """

    pergunta = transcription.lower()

    # --------------------------------------------------
    # 1) VALOR TOTAL VENDIDO NO ANO
    # --------------------------------------------------
    if "faturamento total" in pergunta or "total do ano" in pergunta:
        return (
            f"O faturamento total do ano foi "
            f"R$ {contexto_dados['faturamento_total']:.2f}."
        )

    # --------------------------------------------------
    # 2) QUANTO CADA VENDEDOR VENDEU (UM POR UM)
    # --------------------------------------------------
    if "quanto cada vendedor vendeu" in pergunta or "vendas por vendedor" in pergunta:
        resposta = "Vendas totais por vendedor no ano:\n"
        for vendedor, valor in contexto_dados["total_por_vendedor"].items():
            resposta += f"- {vendedor}: R$ {valor:.2f}\n"
        return resposta

    # --------------------------------------------------
    # 3) QUEM MAIS VENDEU NO ANO
    # --------------------------------------------------
    if "melhor vendedor" in pergunta or "quem mais vendeu no ano" in pergunta:
        melhor = contexto_dados["melhor_vendedor_ano"]
        valor_melhor = contexto_dados["total_por_vendedor"][melhor]

        return (
            f"O melhor vendedor do ano foi {melhor}, "
            f"com um total de R$ {valor_melhor:.2f} em vendas."
        )

    # --------------------------------------------------
    # RESPOSTA PADRÃO (SE NÃO FOR UMA DAS 3 PERGUNTAS)
    # --------------------------------------------------
    return (
        "Ainda não sei responder exatamente essa pergunta sem a API do ChatGPT.\n"
        "Mas o fluxo de gravação, transcrição e análise está funcionando corretamente.\n"
        "Quando a chave OpenAI voltar, essa função será substituída pela versão com GPT."
    )


In [37]:
chatgpt_response = perguntar_chatgpt(transcription, contexto_dados)

print(chatgpt_response)

O faturamento total do ano foi R$ 600000.00.


#5. Sintetizando a Resposta do ChatGPT Como Voz (gTTS) 🔊

In [38]:
!pip install gTTS

In [39]:
from gtts import gTTS
from IPython.display import Audio, display

gtts_object = gTTS(text=chatgpt_response, lang=language, slow=False)

response_audio = "/content/response_audio.mp3"
gtts_object.save(response_audio)

display(Audio(response_audio, autoplay=True))
